# Model Training

### 1. Imports and loading WordNet

In [1]:
import json
import pickle as pkl
import nltk
import random
import numpy as np

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('popular', quiet=True)
lemmatiser = WordNetLemmatizer()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### 2. Loading and organising data

In [2]:
## initialising lists and loading data
words = []
documents = []
intents = json.loads(open('intent_en.json').read())

## getting classes and stopwords
classes = [i['tag'] for i in intents['intents']]
ignore_words = ["!", "@", "#", "$", "%", "*", "?"]

## extracting data

# exporing user input data
for intent in intents['intents']:
    for user_input in intent['patterns']:
        
        # tokenising and adding to list
        word = nltk.word_tokenize(user_input)
        words.extend(word)
        
        # add to document list to identify the intent associated to input
        documents.append((word, intent['tag']))
documents[:3]

[(['Hi'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting')]

In [3]:
# lemmatising words
words = [lemmatiser.lemmatize(w.lower()) for w in words if w not in ignore_words]

# sorting lists
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# saving sorted lists
pkl.dump(words, open('words.pkl', 'wb'))
pkl.dump(classes, open('classes.pkl', 'wb'))

### 3. Initialising training variables

In [4]:
# initialising training
training = []
output_empty = [0] * len(classes)
for document in documents:
    # initializing word bag
    word_bag = []

    # listing user input words
    userinput_words = document[0]

    # lemmatising each word in userinput_words
    userinput_words = [lemmatiser.lemmatize( word.lower()) for word in userinput_words]

    # generating word set with 1, if there's word match in the current pattern 
    for word in words:
        word_bag.append(1) if word in userinput_words else word_bag.append(0)

    # output_row works as a key to the list
    # output = 0 to each intent, and output = 1 to current intent
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([word_bag, output_row])

# shuffling and converting training set to a numpy array
random.shuffle(training)
training = np.array(training, dtype="object")
# generating training data (x is the user input patterns, y is the intents)
x = list(training[:, 0])
y = list(training[:, 1])

### 4. Training model

In [5]:
### Creating model (Simple Multilayer Perceptron)
## First layer: 128 neurons
## Second layer: 64 neurons
## Third layer: neurons equal to number of intents
model = Sequential()
model.add(Dense(128, input_shape=(len(x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))

## Compiling the model using stochastic gradient descent with Nesterov accelerated gradient
## Nesterov Accelerated Gradient (NAG) is used to measure cost function gradient slightly ahead momentum direction instead of local position
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
m = model.fit(np.array(x), np.array(y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', m)

print("done")

Epoch 1/200
95/95 [==============================] - 0s 5ms/step - loss: 3.5927 - accuracy: 0.1081
Epoch 2/200
95/95 [==============================] - 0s 4ms/step - loss: 3.1626 - accuracy: 0.1907
Epoch 3/200
95/95 [==============================] - 0s 5ms/step - loss: 2.8203 - accuracy: 0.2881
Epoch 4/200
95/95 [==============================] - 0s 5ms/step - loss: 2.6463 - accuracy: 0.3178
Epoch 5/200
95/95 [==============================] - 0s 5ms/step - loss: 2.4341 - accuracy: 0.3771
Epoch 6/200
95/95 [==============================] - 0s 4ms/step - loss: 2.2944 - accuracy: 0.4237
Epoch 7/200
95/95 [==============================] - 1s 6ms/step - loss: 2.0676 - accuracy: 0.4746
Epoch 8/200
95/95 [==============================] - 0s 4ms/step - loss: 1.9162 - accuracy: 0.4936
Epoch 9/200
95/95 [==============================] - 0s 5ms/step - loss: 1.7572 - accuracy: 0.5169
Epoch 10/200
95/95 [==============================] - 0s 4ms/step - loss: 1.5911 - accuracy: 0.5720
Epoch 11/

95/95 [==============================] - 1s 6ms/step - loss: 0.1751 - accuracy: 0.9428
Epoch 80/200
95/95 [==============================] - 0s 5ms/step - loss: 0.1777 - accuracy: 0.9492
Epoch 81/200
95/95 [==============================] - 0s 5ms/step - loss: 0.2158 - accuracy: 0.9386
Epoch 82/200
95/95 [==============================] - 0s 4ms/step - loss: 0.2365 - accuracy: 0.9258
Epoch 83/200
95/95 [==============================] - 0s 5ms/step - loss: 0.1751 - accuracy: 0.9364
Epoch 84/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1718 - accuracy: 0.9449
Epoch 85/200
95/95 [==============================] - 0s 5ms/step - loss: 0.1768 - accuracy: 0.9470
Epoch 86/200
95/95 [==============================] - 0s 5ms/step - loss: 0.2042 - accuracy: 0.9258
Epoch 87/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1513 - accuracy: 0.9534
Epoch 88/200
95/95 [==============================] - 0s 5ms/step - loss: 0.1785 - accuracy: 0.9343
Epoch 89/200


95/95 [==============================] - 0s 5ms/step - loss: 0.1551 - accuracy: 0.9555
Epoch 160/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1221 - accuracy: 0.9640
Epoch 161/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1228 - accuracy: 0.9576
Epoch 162/200
95/95 [==============================] - 0s 4ms/step - loss: 0.2420 - accuracy: 0.9258
Epoch 163/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1218 - accuracy: 0.9619
Epoch 164/200
95/95 [==============================] - 0s 4ms/step - loss: 0.0981 - accuracy: 0.9682TA: 0s - loss: 0.1039 - accu
Epoch 165/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1289 - accuracy: 0.9640
Epoch 166/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1197 - accuracy: 0.9597
Epoch 167/200
95/95 [==============================] - 0s 4ms/step - loss: 0.1236 - accuracy: 0.9576
Epoch 168/200
95/95 [==============================] - 0s 4ms/step - loss: 0.